In [2]:
import xarray as xr 
import glob 
import matplotlib.pyplot as plt
from scipy.signal import detrend

# from windspharm.xarray import VectorWind
import torch_harmonics as harmonics
import torch
import numpy as np
import copy

import credit.pol_lapdiff_filt as plf

import xarray as xr 
import os 
import numpy as np 
import sys 
import matplotlib.pyplot as plt 
import glob
import pickle
# from eofs.xarray import Eof

import os
import cartopy.crs as ccrs

import numpy as np
import os
from sklearn.preprocessing import QuantileTransformer
from functools import lru_cache

import math
import copy
from pathlib import Path
from random import random
from functools import partial
from collections import namedtuple
from multiprocessing import cpu_count

from scipy.optimize import linear_sum_assignment

from PIL import Image
from tqdm.auto import tqdm

import matplotlib.pyplot as plt

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.colors as mcolors

## Open Zarr

In [35]:
conc = []
for yryr in range(1979,2015):
    DSdo_1 = xr.open_zarr(f'/glade/campaign/cisl/aiml/wchapman/MLWPS/STAGING/f.e21.CREDIT_climate_{yryr}_ERA5scaled_F32.zarr')
    conc.append(DSdo_1)

DSzarr1 = xr.concat(conc, dim='time')

## Stream Function from ML Run using Torch Harmonics

In [16]:
lev_meta = 20
LevDo = int(DSzarr1.level[lev_meta].values)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
FNS = sorted(glob.glob('/glade/derecho/scratch/wchapman/CREDIT/cesm_wxformer_00191/1981-01-02T00Z/1D/averaged_UVTQtotPSPRECTTREFHT*'))
DSs = xr.open_mfdataset(FNS)
nlat = DSs.sizes['latitude']
nlon = DSs.sizes['longitude']
ntime = DSs.sizes['time']

# Initialize Diffusion and Pole Filter
DPF = plf.Diffusion_and_Pole_Filter(nlat=nlat, nlon=nlon, device=device)

# Initialize lists to store results for all times
streamfunc_list = []
velocity_potential_list = []
U_rot_list = []
V_rot_list = []
vorticity_list = []
divergence_list = []

# Loop over all time steps
for t in range(ntime):
    if (t + 1) % 100 == 0:
        print(f'Processing timestep {t + 1}/{ntime}')
    
    # Extract U and V at each time step
    U = DSs['U'].isel(time=t).sel(level=lev_meta).values
    V = DSs['V'].isel(time=t).sel(level=lev_meta).values
    
    # Convert to Torch tensors
    U = torch.tensor(U).clone().detach().to(device)
    V = torch.tensor(V).clone().detach().to(device)
    
    # Stack U and V
    ugrid = torch.stack((U, V)).to(device)
    
    # Compute vorticity and divergence
    vrt, div = DPF.vrtdivspec(ugrid)
    
    # 1. Compute Stream Function
    streamfunc_spec = - DPF.invlap * vrt
    streamfunc_spec *= torch.exp(-0.01 * (DPF.lap)**2)  # Smoothing
    streamfunc = DPF.spec2grid(streamfunc_spec)
    
    # 2. Compute Velocity Potential
    velocity_potential_spec = DPF.invlap * div
    velocity_potential_spec *= torch.exp(-0.01 * (DPF.lap)**2)  # Smoothing
    velocity_potential = DPF.spec2grid(velocity_potential_spec)
    
    # 3. Compute Rotational Wind Components (U_rot, V_rot)
    U_rot, V_rot = DPF.getuv(torch.stack((streamfunc_spec, torch.zeros_like(streamfunc_spec)), dim=0))
    
    # Store results (detach and move to CPU for xarray compatibility)
    streamfunc_list.append(streamfunc.cpu().detach())
    velocity_potential_list.append(velocity_potential.cpu().detach())
    U_rot_list.append(U_rot.cpu().detach())
    V_rot_list.append(V_rot.cpu().detach())
    
    # Store vorticity and divergence
    vorticity_list.append(DPF.spec2grid(vrt.cpu().detach()))
    divergence_list.append(DPF.spec2grid(div.cpu().detach()))

# Stack results along time dimension
streamfunc_stack = torch.stack(streamfunc_list)
velocity_potential_stack = torch.stack(velocity_potential_list)
U_rot_stack = torch.stack(U_rot_list)
V_rot_stack = torch.stack(V_rot_list)
vorticity_stack = torch.stack(vorticity_list)
divergence_stack = torch.stack(divergence_list)

# Convert to xarray DataArrays with appropriate dimensions
streamfunc_da = xr.DataArray(
    streamfunc_stack.numpy(),
    dims=['time', 'latitude', 'longitude'],
    coords={'time': DSs['time'], 'latitude': DSs['latitude'], 'longitude': DSs['longitude']},
    name='stream_function'
)

velocity_potential_da = xr.DataArray(
    velocity_potential_stack.numpy(),
    dims=['time', 'latitude', 'longitude'],
    coords={'time': DSs['time'], 'latitude': DSs['latitude'], 'longitude': DSs['longitude']},
    name='velocity_potential'
)

U_rot_da = xr.DataArray(
    U_rot_stack.numpy(),
    dims=['time', 'latitude', 'longitude'],
    coords={'time': DSs['time'], 'latitude': DSs['latitude'], 'longitude': DSs['longitude']},
    name='U_rot'
)

V_rot_da = xr.DataArray(
    V_rot_stack.numpy(),
    dims=['time', 'latitude', 'longitude'],
    coords={'time': DSs['time'], 'latitude': DSs['latitude'], 'longitude': DSs['longitude']},
    name='V_rot'
)

vorticity_da = xr.DataArray(
    vorticity_stack.numpy(),
    dims=['time', 'latitude', 'longitude'],
    coords={'time': DSs['time'], 'latitude': DSs['latitude'], 'longitude': DSs['longitude']},
    name='vorticity'
)

divergence_da = xr.DataArray(
    divergence_stack.numpy(),
    dims=['time', 'latitude', 'longitude'],
    coords={'time': DSs['time'], 'latitude': DSs['latitude'], 'longitude': DSs['longitude']},
    name='divergence'
)

# Merge into a single dataset and save
DSmerged = xr.merge([streamfunc_da, velocity_potential_da, U_rot_da, V_rot_da, vorticity_da, divergence_da])
DSmerged.to_netcdf(f'/glade/derecho/scratch/wchapman/CREDIT/cesm_wxformer_00191/1981-01-02T00Z/1D/SF_VP_{LevDo}mb_and_associates.nc')

Processing timestep 100/10317
Processing timestep 200/10317
Processing timestep 300/10317
Processing timestep 400/10317
Processing timestep 500/10317
Processing timestep 600/10317
Processing timestep 700/10317
Processing timestep 800/10317
Processing timestep 900/10317
Processing timestep 1000/10317
Processing timestep 1100/10317
Processing timestep 1200/10317
Processing timestep 1300/10317
Processing timestep 1400/10317
Processing timestep 1500/10317
Processing timestep 1600/10317
Processing timestep 1700/10317
Processing timestep 1800/10317
Processing timestep 1900/10317
Processing timestep 2000/10317
Processing timestep 2100/10317
Processing timestep 2200/10317
Processing timestep 2300/10317
Processing timestep 2400/10317
Processing timestep 2500/10317
Processing timestep 2600/10317
Processing timestep 2700/10317
Processing timestep 2800/10317
Processing timestep 2900/10317
Processing timestep 3000/10317
Processing timestep 3100/10317
Processing timestep 3200/10317
Processing timest

## For CESM

In [ ]:
lev_meta = 20
LevDo = int(DSzarr1.level[lev_meta].values)

print('... taking daily mean...')
DSzarr1 = DSzarr1[['U','V']].sel(level=LevDo, method='nearest').resample(time='1D').mean()
print('...loading...')
DSzarr1 = DSzarr1.load()
print('... done taking daily mean...')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

nlat = DSzarr1.sizes['latitude']
nlon = DSzarr1.sizes['longitude']
ntime = DSzarr1.sizes['time']

# Initialize Diffusion and Pole Filter
DPF = plf.Diffusion_and_Pole_Filter(nlat=nlat, nlon=nlon, device=device)

# Initialize lists to store results for all times
streamfunc_list = []
velocity_potential_list = []
U_rot_list = []
V_rot_list = []
vorticity_list = []
divergence_list = []

# Loop over all time steps
for t in range(ntime):
    if (t + 1) % 100 == 0:
        print(f'Processing timestep {t + 1}/{ntime}')
    
    # Extract U and V at each time step
    U = DSzarr1['U'].isel(time=t).values
    V = DSzarr1['V'].isel(time=t).values
    
    # Convert to Torch tensors
    U = torch.tensor(U).clone().detach().to(device)
    V = torch.tensor(V).clone().detach().to(device)
    
    # Stack U and V
    ugrid = torch.stack((U, V)).to(device)
    
    # Compute vorticity and divergence
    vrt, div = DPF.vrtdivspec(ugrid)
    
    # 1. Compute Stream Function
    streamfunc_spec = - DPF.invlap * vrt
    streamfunc_spec *= torch.exp(-0.01 * (DPF.lap)**2)  # Smoothing
    streamfunc = DPF.spec2grid(streamfunc_spec)
    
    # 2. Compute Velocity Potential
    velocity_potential_spec = DPF.invlap * div
    velocity_potential_spec *= torch.exp(-0.01 * (DPF.lap)**2)  # Smoothing
    velocity_potential = DPF.spec2grid(velocity_potential_spec)
    
    # 3. Compute Rotational Wind Components (U_rot, V_rot)
    U_rot, V_rot = DPF.getuv(torch.stack((streamfunc_spec, torch.zeros_like(streamfunc_spec)), dim=0))
    
    # Store results (detach and move to CPU for xarray compatibility)
    streamfunc_list.append(streamfunc.cpu().detach())
    velocity_potential_list.append(velocity_potential.cpu().detach())
    U_rot_list.append(U_rot.cpu().detach())
    V_rot_list.append(V_rot.cpu().detach())
    
    # Store vorticity and divergence
    vorticity_list.append(DPF.spec2grid(vrt.cpu().detach()))
    divergence_list.append(DPF.spec2grid(div.cpu().detach()))

# Stack results along time dimension
streamfunc_stack = torch.stack(streamfunc_list)
velocity_potential_stack = torch.stack(velocity_potential_list)
U_rot_stack = torch.stack(U_rot_list)
V_rot_stack = torch.stack(V_rot_list)
vorticity_stack = torch.stack(vorticity_list)
divergence_stack = torch.stack(divergence_list)

# Convert to xarray DataArrays with appropriate dimensions
streamfunc_da = xr.DataArray(
    streamfunc_stack.numpy(),
    dims=['time', 'latitude', 'longitude'],
    coords={'time': DSzarr1['time'], 'latitude': DSzarr1['latitude'], 'longitude': DSzarr1['longitude']},
    name='stream_function'
)

velocity_potential_da = xr.DataArray(
    velocity_potential_stack.numpy(),
    dims=['time', 'latitude', 'longitude'],
    coords={'time': DSzarr1['time'], 'latitude': DSzarr1['latitude'], 'longitude': DSzarr1['longitude']},
    name='velocity_potential'
)

U_rot_da = xr.DataArray(
    U_rot_stack.numpy(),
    dims=['time', 'latitude', 'longitude'],
    coords={'time': DSzarr1['time'], 'latitude': DSzarr1['latitude'], 'longitude': DSzarr1['longitude']},
    name='U_rot'
)

V_rot_da = xr.DataArray(
    V_rot_stack.numpy(),
    dims=['time', 'latitude', 'longitude'],
    coords={'time': DSzarr1['time'], 'latitude': DSzarr1['latitude'], 'longitude': DSzarr1['longitude']},
    name='V_rot'
)

vorticity_da = xr.DataArray(
    vorticity_stack.numpy(),
    dims=['time', 'latitude', 'longitude'],
    coords={'time': DSzarr1['time'], 'latitude': DSzarr1['latitude'], 'longitude': DSzarr1['longitude']},
    name='vorticity'
)

divergence_da = xr.DataArray(
    divergence_stack.numpy(),
    dims=['time', 'latitude', 'longitude'],
    coords={'time': DSzarr1['time'], 'latitude': DSzarr1['latitude'], 'longitude': DSzarr1['longitude']},
    name='divergence'
)

# Merge into a single dataset and save
DSmerged = xr.merge([streamfunc_da, velocity_potential_da, U_rot_da, V_rot_da, vorticity_da, divergence_da])
DSmerged.to_netcdf(f'/glade/derecho/scratch/wchapman/CREDIT/cesm_wxformer_00191/1981-01-02T00Z/1D/SF_VP_CESM_{LevDo}mb_and_associates.nc')

... taking daily mean...
... done taking daily mean...
Processing timestep 10/13140
Processing timestep 20/13140
Processing timestep 30/13140
Processing timestep 40/13140
Processing timestep 50/13140
Processing timestep 60/13140
Processing timestep 70/13140
Processing timestep 80/13140
Processing timestep 90/13140
Processing timestep 100/13140
Processing timestep 110/13140
Processing timestep 120/13140
Processing timestep 130/13140
Processing timestep 140/13140
Processing timestep 150/13140
Processing timestep 160/13140
Processing timestep 170/13140
Processing timestep 180/13140
Processing timestep 190/13140
Processing timestep 200/13140
Processing timestep 210/13140
Processing timestep 220/13140
Processing timestep 230/13140
Processing timestep 240/13140
Processing timestep 250/13140
Processing timestep 260/13140
Processing timestep 270/13140
Processing timestep 280/13140
Processing timestep 290/13140
Processing timestep 300/13140
Processing timestep 310/13140
Processing timestep 320/